# Problem 1 (12 points)

Recall the word ladder puzzle from Homework 1, in which vowel changes cost twice those of consonant changes. You implemented best-first search as a general procedure for solving this problem. Suppose now that we do not maintain a ```reached``` table and that children of expanded nodes are always added to the frontier.

1. Explain how this change affects the property of completeness. Remember that it is possible for a problem to have no solution. Consider each of the following algorithms: depth-first search, breadth-first search, uniform-cost search, and A* search with the simple Hamming distance heuristic.

2. Suppose that multiple solutions exist for a given problem. Without a reached table, which of the above four algorithms are guaranteed to return an optimal solution, and which are not? Explain for each of them.

3. Explain whether the "simple" Hamming distance and the Hamming distance with vowels heuristics are still admissible when A* is implemented without a reached table.

ENTER YOUR RESPONSES HERE

1. A searching algorithm is complete if the search tree is finite. All these four algotirhms are complete in the original setting. If we do not maintain a `reached` table, the only change is that there will be some loop in our searching route, i.e., some path in the search tree can be something like "let" -> "get" -> "let".
   - depth-first search: not complete, since the search tree will be infinite due to the paths corresponding to a loop.
   - breadth-first search: complete, if it has a solution, since if the shallowest goal node is at depth `d`, it will eventually find it after expanding all the nodes shallower than `d`, with a finite braching factor. Even if there exist some looping path, instead of continuing down to exhaust its depth, we first search through some other (correct and non-looping) paths.
   - uniform-cost search: complete, if it has a solution. Similar as BFS, if there is a goal node with the least finite cost `c`, UCS will eventually find it after expanding all the nodes with cost lower than `c`, which is also finite since we always increase the `cost` of the new state to make it higher then that of the old node, and, we have a finite branching factor.
   - A* search with the simple Hamming distance heuristic: complete, if it has a solution. Similar as BFS and UCS, except that here the `cost` includes the simple Hamming distance heuristic.

2. - depth-first search: not optimal, since the search tree will be infinite due to the paths corresponding to a loop.
   - breadth-first search: not optimal. BFS is optimal if all operators (i.e., arcs) have the same constant cost, or costs are positive, non-decreasing with depth. However, the rule that vowel changes cost twice those of consonant changes will violate this premise.
   - uniform-cost search: optimal. Unlike BFS which expends by depth, UCS explores increasing cost contours, so the first encountered solution must also be the optima.
   - A* search with the simple Hamming distance heuristic: optimal. A* is optimal if the heuristic is admissible; in our case, the Hamming distance heuristic is indeed admissible.

3. Both are still admissible, since when A* is implemented without a reached table, the cost of the optimal solution will not be changed. If there is  𝑥  characters diffrent between the current state and the goal, we need to change at least  𝑥  characters to achieve the solution, so cost of the optimal solution is at least  𝑥 

# Problem 2 (16 points)

We are computing the value $v_0$ of a chance node $C_0$ in a game tree. $C_0$ has children $C_1, C_2, ..., C_n$, occurring with probability $p_1, p_2, ..., p_n$, respectively. We have already retrieved the values of the first $k$ children: $v_1, ..., v_k$. We have not yet seen the values of the remaining children nodes. Give an expression for the maximum possible value $v_0$, if the maximum possible value for any child node is $v_{\text{max}}$.

ENTER YOUR EXPRESSION HERE

For chance nodes, node value is the expected value over all its children. \\
$v_{0\ max} = \sum_{i=k}^{n}p_{i}v_{max} + \sum_{i=1}^{k}p_{i}v_i$


X and O are playing 3x3 tic-tac-toe. Instead of playing optimally, O now plays randomly, choosing its move according to known probabilities each turn. X's strategy is implemented in an ```X_value``` function (signature shown below). You also have access to functions returning O's ```actions``` in a given state and the ```result``` of taking an action from a given state.

```
def X_value(state):
  INPUTS: A game state
  OUTPUT: Value of state assuming it is X's turn to move

def actions(state):
  INPUT: A game state during O's turn
  OUTPUTS: List of (action, probability) tuples

def result(state, action):
  INPUTS: A game state and action
  OUTPUT: New game state as a result of O taking action from state
```

Calling the above functions as necessary, complete the ```O_value``` function below, which returns the value of the given game state assuming it is O's turn to move. As the function loops through the children states to compute the expected value, it should also prune when possible. ```O_value``` should stop retrieving the values of its remaining children nodes as soon as it recognizes that it is no longer possible to return a value greater than ```alpha``` (it can simply return the value computed so far).

As a hint, you should be using the expression you derived above. You may assume that ```V_MAX``` is a known upper bound on state utility values and accessible in your implementation.

In [ ]:
def O_value(state, alpha):
  isTerminal, score = terminal(state)
  if isTerminal:
    return score

  # YOUR CODE HERE
  v = V_MAX
  acts = actions(state)
  for a, p in acts:
    new_state = result(state, a)
    v2 = X_value(new_state)
    v -= (V_MAX-v2)*p
    if v <= alpha:
      return v
  return v


You can check that your implementation above is free of syntactical and logical errors by running the unit test below (note this does *not* give any indication of the correctness of your implementation). We cannot answer questions on what the expected output should be.

In [ ]:
def terminal(s):
  return False, None
def actions(s):
  return [('a1',0.5), ('a2',0.5)]
def result(s,a):
  if a == 'a1': return 's1'
  elif a == 'a2': return 's2'
def X_value(s):
  if s == 's1': return 1
  elif s == 's2': return 1

V_MAX = 1
print(O_value('s0',1))

1.0


# Problem 3 (20 points)

Recall that $TD(0)$ solves the prediction problem of evaluating a fixed policy $\pi$ using temporal-difference updates to estimated state values. The update to the value $V^\pi(s)$ is given by

$$ V^\pi(s) \leftarrow V^\pi(s) + \alpha (r + \gamma V^\pi(s') - V^\pi(s)), $$

where $r$ is the observed reward, $s'$ is the successor state, $\alpha$ is the learning rate, and $\gamma$ is the discount factor.

Suppose that our agent has traversed a sequence of states $s_1, s_2, ..., s_n$ (following $\pi$) and observed the corresponding rewards $r_1, r_2, ..., r_{n-1}$. All distinct states in the problem have been encountered at least once prior to $s_n$. The underlying transition and reward functions map $(s,a,s')$ inputs to fixed probabilities and real values, respectively. 

1. Provide a set of equations in terms of the observed states, rewards, and problem parameters that, if solvable, produces the values to all states $V^\pi(s)$ such that all TD updates using the observed state-reward sequences are zero. It is acceptable for your set to contain possibly redundant equations.

2. How many unknowns are there in your equations? Explain whether it is possible to have more *unique* equations than unknowns and why that may occur.



ENTER YOUR RESPONSES HERE

1. $r_{1} + \gamma V^\pi(s_{2}) = V^\pi(s_{1})$  

   $r_{2} + \gamma V^\pi(s_{3}) = V^\pi(s_{2})$ 

   ... 

   $r_{n-1} + \gamma V^\pi(s_{n}) = V^\pi(s_{n-1})$

2. n unknowns are there in my equations: $s_1, \dots, s_{n}$ if $ \gamma$ is also given; otherwisse, n+1. it is possible to have more unique equations than unknowns, since there might be duplicate states among $s_1, \dots, s_{n}$, e.g., there are only three distinct states $A, B, C$, and $s_1 = s_{3} = A $.

As with policy evaluation using a known model, an alternative to solving the equations that you wrote is dynamic programming, this time using the TD update rather than Bellman update. Starting with an initialized dictionary of values ```V``` of the form ```{state:value}```, this scheme should sweep over the lists of observed ```states``` and ```rewards``` sequences, performing a TD update to  ```V``` for each state-reward pair seen. This sweep should then be repeated until the maximum absolute change for a value is smaller than the provided ```threshold```. (You may assume that the provided inputs will produce a solution that converges.)

Implement the described algorithm in ```TD0``` below. In addition to the described inputs, note that we also have the ```alpha``` and ```gamma``` parameters.

In [ ]:
def TD0(V, states, rewards, alpha, gamma, threshold):
  max_diff = float("inf")
  while max_diff >= threshold:
    # YOUR CODE HERE    
    max_diff = 0
    for i in range(len(rewards)):
       delta = alpha*(rewards[i] + gamma*V[states[i+1]] - V[states[i]]) 
       V[states[i]] += delta
       if delta > max_diff:
         max_diff = delta

  return V

You can check that your implementation above is free of syntactical and logical errors by running the unit test below (note this does *not* give any indication of the correctness of your implementation). We cannot answer questions on what the expected output should be.

In [ ]:
states = ['s1', 's2', 's1']
rewards = [1, 1]
alpha = 0.5
gamma = 0.8
threshold = 1e-6

V = {'s1':0, 's2':0}
print(TD0(V, states, rewards, alpha, gamma, threshold))

{'s1': 4.9999935821469315, 's2': 4.999994000289807}


# Problem 4 (16 points)

We have a Naive Bayes model with class variable $Y$ and feature variables $F_1, ..., F_n$. Suppose we observe feature $F_e = f_e$. 

1.  Give an expression for $\Pr(F_q \mid f_e)$, the distribution of the query feature $F_q$ given the evidence feature $f_e$. You may also provide your answer in the form of an unnormalized distribution. All quantities in your expression should be contained in the set of Naive Bayes parameters.

2.  Briefly describe how you would estimate $\Pr(F_q \mid f_e)$ using likelihood weighting instead. What variables should be sampled, what order are they sampled in, and what sampling distributions are used? Also be sure to describe how the desired distribution is obtained from samples.

ENTER YOUR RESPONSES HERE

1.  $\Pr(F_q \mid f_e) = \frac{\sum_y\Pr(F_q, y, f_e)}{\Pr(f_e)} = \frac{\sum_y\Pr(F_q | y) \Pr(y , f_e) }{\Pr(f_e)} = \frac{\sum_y\Pr(F_q | y) \Pr(f_e |y) \Pr(y) }{\Pr(f_e)}$

2.  Fix evidence variables $F_e = f_e$, sample only nonevidence variable $Y, F_q$ ($Y$ first), and weight each sample by the likelihood it accords the evidence. Sampling distributions: \\
$ S_{WE}(F_q, f_e)w(F_q, f_e) = Pr(F_q | y) \Pr(f_e |y)$ \\
To extract the distribution of the query variable, the sum of the weights of the samples where $F_q = f_q$ for each of $F_q$'s possible value, and then do a normalization at last.

Use the expression that you came up with in 1. to implement ```feature_likelihood``` below. The function inputs are as follows:
*   ```prior``` is a 1D numpy array containing the distribution $\Pr(Y)$.
*   ```fe_given_y``` is a 1D numpy array containing the probabilities $\Pr(f_e \mid Y)$.
*   ```fq_given_y``` is a 2D numpy array, in which ```fq_given_y[i,j]``` is the probability $\Pr(F_q = i \mid Y = j)$.

Make sure that your outputs are properly normalized.

In [ ]:
import numpy as np

def feature_likelihood(prior, fe_given_y, fq_given_y):
  # YOUR CODE HERE
  len_q, len_y = fq_given_y.shape
  ret = np.zeros(len_q)
  for i in range(len_y):
    ret += fq_given_y[:,i]*fe_given_y[i]*prior[i]
  
  return np.divide(ret, np.sum(ret))


You can check that your implementation above is free of syntactical and logical errors by running the unit test below (note this does *not* give any indication of the correctness of your implementation). We cannot answer questions on what the expected output should be.

In [ ]:
prior = np.array([0.5, 0.5])
fe_given_y = np.array([0.7, 0.6])
fq_given_y = np.array([[0.3, 0.6], [0, 0.4], [0.7, 0]])

print(feature_likelihood(prior, fe_given_y, fq_given_y))

[0.43846154 0.18461538 0.37692308]
